# Ćwiczenia 6. Walidacja Krzyżowa

## PyTorch na następne ćwiczenia.

Proszę zainstalować pakiet [PyTorch](https://pytorch.org/) oraz torchvision na kolejne zajęcia. Jeśli używane, mając swoje środowisko aktywne użyć:

 * GPU: `conda install pytorch torchvision cudatoolkit=9.0 -c pytorch`
 * tylko CPU: `conda install pytorch torchvision cpuonly  -c pytorch`

## Klasyfikacja

Dzisiaj na zajęciach zajmiemy się problemem klasyfikacji. Podobnie do regresji liniowej jest to przykład uczenia nadzorowanego, ale zamiast przewidywać konkretną liczbę dla danej obserwacji, przewidujemy jego przynajeżność do jednej z *k* klas. Na tych zajęciach będziemy rozważać klasyfikacje binarną, czyli uczyć modele odpowiadające funkcji:

$$ f(x) = y, \quad y \in \{0,1\} $$

Poniżej ładowane są dane, do razu już podzielone na dwie części.

In [2]:
import numpy as np
from utils import get_data

X_train, X_test, y_train, y_test = get_data()

## Zadanie 1.1 (0.5 pkt.)

Używając modelu [`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) z pakietu sklearn uzyskać 100% dokładność (mierzoną miarą [`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html))na zbiorze treningowym. Państwa zadanie polega na dobraniu parametru `gamma`, dla ułatwienia proszę nie zmieniać pozostałych domyślnych parametrów. Zalecany przedział parametru `gamma` to wartości z przedziału [0, 1] w skali logarytmicznej.

In [3]:
from sklearn.svm import SVC

gammas = [10 ** (-i) for i in range(10)]
train_accs = []
for gamma in gammas:
    svm = SVC(gamma=gamma)
    svm.fit(X_train, y_train)
    train_acc = svm.score(X_train, y_train)
    train_accs.append(train_acc)
best_gamma = gammas[np.argmax(train_accs)]
print(best_gamma)

1


In [4]:
# test
best_gamma = best_gamma

svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
train_acc = svm.score(X_train, y_train)
assert train_acc == 1.

## Zadanie 1.2 (0.5 pkt.)
Używając tej samej rodziny modeli znajdź tym razem taką wartość `gamma`, która daje najlepszy wynik na zbiorze **testowym**. Powinieneś(-aś) być w stanie osiągnąć wynik co najmniej `0.95` accuracy. 

In [5]:
from sklearn.svm import SVC

gammas = [10 ** (-i) for i in range(10)]
train_accs = []
for gamma in gammas:
    svm = SVC(gamma=gamma)
    svm.fit(X_train, y_train)
    train_acc = svm.score(X_test, y_test)
    train_accs.append(train_acc)
best_gamma = gammas[np.argmax(train_accs)]
print(best_gamma)

0.0001


In [6]:
# test
best_gamma = best_gamma

svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
test_acc = svm.score(X_test, y_test)
assert test_acc >= 0.95

### Problem.

**W poprzednim zadaniu zakładaliśmy, że podział na zbiór trenujący/testujący jest nam podany z góry, ale co jeśli go nie mamy?**

Możemy oczywiście sami arbitralnie wybrać część datasetu i uznać ją za nasz zbiór testowy, ale to mogą się z tym wiązać dodatkowe problemy: co jeśli wybrany przez nas fragment jest akurat różny od reszty datasetu, lub odwrotnie?

**Rozwiązanie:** Walidacja Krzyżowa.

1. Podziel dataset na zadaną przez parametr `k` liczbę (prawie) równych grup.
2. Dla każdego podziału, zwróć jedną z tych części jako zbiór testowy, a sumę reszty jako zbiór treningowy.
3. Po nauczeniu łącznie `k` modeli, uśrednij ich wyniki na zbiorach testowych i zwróć jako ostateczny wynik.

## Zadanie 2. (2 pkt.)

Państwa zadaniem jest zaimplementowanie walidacji krzyżowej, czyli funkcji, która dla podanego datasetu w postaci macierzy danych `X` i wektora etykiet `y` zwróci listę `k` krotek: 
  
  `(treningowe_dane, treningowe_etykiety, testowe_dane, testowe_etykiety)`
  
podziałów dokonanych przez walidację krzyżową. Następnie należy użyć modelu z poprzedniego zadania dla policzenia dokładności na zbiorze testowym dla walidacji krzyżowej.

Proszę **nie** korzystać z gotowych rozwiązań dostępnych w pakiecie sklearn.


In [59]:
from typing import List, Tuple
import math

def cross_validation(X: np.ndarray, y: np.ndarray, k: int) -> List[Tuple[np.ndarray, np.ndarray, 
                                                                         np.ndarray, np.ndarray]]:
    part_len = math.ceil(len(X) / k)
    X_parts = np.asarray([X[i*part_len:(i+1)*part_len] for i in range(k-1)] + [X[(k-1)*part_len:]])
    y_parts = np.asarray([y[i*part_len:(i+1)*part_len] for i in range(k-1)] + [y[(k-1)*part_len:]])
    result = []
    for i in range(k):
#         print(np.append(X_parts[:i], X_parts[i+1:])[0])
        result.append((np.concatenate(np.append(X_parts[:i], X_parts[i+1:], axis=0)), np.concatenate(np.append(y_parts[:i], y_parts[i+1:], axis=0)), X_parts[i], y_parts[i]))
    return result

In [60]:
from checker import test_cv

test_cv(cross_validation)

In [19]:
X, y = get_data(split=False)
k = 5
data_groups = cross_validation(X, y, k)

cv_accuracy = 0
for data_group in data_groups:
    svm = SVC(gamma=best_gamma)
    svm.fit(data_group[0], data_group[1])
    test_acc = svm.score(data_group[2], data_group[3])
    cv_accuracy += test_acc
    
cv_accuracy = cv_accuracy / k
print(cv_accuracy)

(0,)
(1,)
(2,)
(3,)
(4,)
0.8946281633286756


## Zadanie 3 (1 pkt.)

Mając już lepszą metodę walidacji naszego rozwiązania Państwa zadaniem jest znalezienia najlepszego zestawu hiperparametrów dla modelu z poprzednich zadań, lecz tym razem będziemy szukać również parametru `C`. Parametru C zaleca się szukać w przedziale $(0, + \infty)$ również w skali logarytmicznej.

W zadaniu należy oczywiście skorzystać z zaimplementowanej przez Państwa walidacji krzyżowej. Ponieważ dla dwóch parametrów `C` oraz `gamma` możliwych kombinacji do przetestowania robi są dość sporo dla przetestowania dużych zakresów zalecane są również inne metody przeszukiwania, takie jak:

* przeszukiwanie po kolei z jednym z parametrów ustalonym na stałą.
* przeszukiwanie losowe obu parametrów

Oczywiście jeśli zasoby na to pozwalają można szukać tzw. "grid searchem".

Powinno udać się Państwu wyciągnąć przynajmniej `0.94` accuracy na walidacji krzyżowej.

In [21]:
from sklearn.svm import SVC

X, y = get_data(split=False)

gammas = [10 ** (-i) for i in range(10)]
Cs = [10 ** i for i in range(-2,8)]
data_groups = cross_validation(X, y, k)
accs = []
for gamma in gammas:
    for C in Cs:
        valid_acc = 0
        for data_group in data_groups:
            svm = SVC(gamma=gamma, C=C)
            svm.fit(data_group[0], data_group[1])
            valid_acc += svm.score(data_group[2], data_group[3])
        accs.append(valid_acc / k)
best_gamma = gammas[np.argmax(accs)//10]
best_C = Cs[np.argmax(accs) % 10]
print(best_gamma)
print(best_C)
print(np.max(accs))

(0,)
(1,)
(2,)
(3,)
(4,)
1e-07
1000000
0.9525694767893185


## Zadanie 4. (3 punkty)

Załóżmy, że naszym problemem jest zdecydowanie, która rodzina modeli *SVM* najlepiej radzi sobei z naszym datasetem. Przez rodzinę rozumiemy tutaj modele SVM z różną *funkcją bazową* (zwaną często *funkcją jądra*). W pakiecie mamy dostępne kilka możliwości, włącznie z podawaniem swoich własnych, ale w tym zadaniu skupimy się na czterech dostępnych od ręki: `linear`, `poly`, `rbf`, `sigmoid`.

Wiemy jak znaleźć najlepszy zestaw parametrów dla danej rodziny modeli, zrobiliśmy to do tej pory dla domyślnej funkcji bazowej czyli `rbf`. Jak jednak mamy "uczciwie" porównać wyniki modeli pomiędzy sobą? Do tej pory patrzyliśmy na wyniki modelu dla datasetu testowego i to na podstawie tego wyniku wybieraliśmy najlepsze hiperparametry. Jakie mogą być z tym problemy? Overfitting?

Rozwiązanie: jeszcze jedna walidacja krzyżowa!

1. Pierwsza walidacja krzyżowa podzieli nam nasz zbiór na treningowy i testowy. Te testowe zbiory będą naszymi ostatecznymi zbiorami testowymi, na których nie będziemy w żaden sposób się uczyć czy szukać hiperparametrów. 
2. Następnie nasz zbiór treningowy podzielimy ponownie walidacją krzyżową na dwie części: faktyczny treningowy i walidacyjny. Tych dwóch podziałów będziemy używać jak poprzednio do uczenia modelu i testowania hiperparametrów.
3. Po znalezieniu najlepszego zestawu hiperparametrów nauczymy ostatecznie nasz model na sumie zbiorów treningowego i walidacyjnego i sprawdzimy jego dokładność na ostatecznym zbiorze testowym.


**Uwaga**: parametr `C` używany jest dla każdej możliwej funkcji bazowej. Proszę sprawdzić jakie parametry są używane dla jakich funkcji jądra. 
**Hint**: parametry, które mogą państwa interesować to oczywiście `kernel`, oraz `C`, `degree`, `gamma`, `coef0`.

In [62]:
from sklearn.svm import SVC

X, y = get_data(split=False)
k=5
rest_and_test = cross_validation(X, y, k)
train_and_valid = [cross_validation(x[0], x[1], k) for x in rest_and_test]
#rbf
gammas = [10 ** (-i) for i in range(10)]
Cs = [10 ** i for i in range(-2,8)]
global_accs = []
for gamma in gammas:
    for C in Cs:
        acc = 0
        for dataset in train_and_valid:
            valid_acc = 0
            for train_data, train_labels, valid_data, valid_labels in dataset:
                svm = SVC(gamma=gamma, C=C)
                svm.fit(train_data, train_labels)
                valid_acc += svm.score(valid_data, valid_labels)
            acc += valid_acc / k
        global_accs.append(acc / k)
best_gamma = gammas[np.argmax(global_accs)//10]
best_C = Cs[np.argmax(global_accs) % 10]
test_acc = 0
for data, labels, test_data, test_labels in rest_and_test:
    svm = SVC(gamma=best_gamma, C=best_C)
    svm.fit(data, labels)
    test_acc += svm.score(test_data, test_labels)
test_acc /= k
print(best_gamma)
print(best_C)
print(test_acc)

1e-07
100000
0.9507840397453812


In [66]:
#linear
Cs = [10 ** i for i in range(-2,8)]
global_accs = []
for C in Cs:
    acc = 0
    for dataset in train_and_valid:
        valid_acc = 0
        for train_data, train_labels, valid_data, valid_labels in dataset:
            svm = SVC(kernel='linear', C=C, max_iter=100_000)
            svm.fit(train_data, train_labels)
            valid_acc += svm.score(valid_data, valid_labels)
        acc += valid_acc / k
    global_accs.append(acc / k)
best_C = Cs[np.argmax(global_accs)]
test_acc = 0
for data, labels, test_data, test_labels in rest_and_test:
    svm = SVC(kernel='linear', C=best_C)
    svm.fit(data, labels)
    test_acc += svm.score(test_data, test_labels)
test_acc /= k
print(best_C)
print(test_acc)

C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\U

C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\U

C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\U

C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\U

C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\U

C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\U

C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\pawli\.conda\envs\ml19\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\U

0.1
0.9455053563111318


In [74]:
#poly
import warnings
warnings.filterwarnings('ignore', 'Solver terminated early.*')
gammas = [10 ** (-i) for i in range(10)]
coef0s = range(1,11)
degrees = range(1,11)
global_accs = []
for gamma in gammas:
    for coef0 in coef0s:
        for degree in degrees:
            acc = 0
            for dataset in train_and_valid:
                valid_acc = 0
                for train_data, train_labels, valid_data, valid_labels in dataset:
                    svm = SVC(kernel='poly', gamma=gamma, coef0=coef0, degree=degree, max_iter=10_000)
                    svm.fit(train_data, train_labels)
                    valid_acc += svm.score(valid_data, valid_labels)
                acc += valid_acc / k
            global_accs.append(acc / k)
best_gamma = gammas[np.argmax(global_accs) // 100 ]
best_coef0 = coef0s[np.argmax(global_accs) % 100 // 10]
best_degree = degrees[np.argmax(global_accs) % 10]
test_acc = 0
for data, labels, test_data, test_labels in rest_and_test:
    svm = SVC(kernel='poly', gamma=best_gamma, coef0=best_coef0, degree=best_degree)
    svm.fit(data, labels)
    test_acc += svm.score(test_data, test_labels)
test_acc /= k
print(best_gamma)
print(best_coef0)
print(best_degree)
print(test_acc)

1e-08
7
7
0.9507840397453812


In [76]:
#sigmoid
gammas = [10 ** (-i) for i in range(10)]
coef0s = range(1,11)
Cs = [10 ** i for i in range(-2,8)]
global_accs = []
for gamma in gammas:
    for coef0 in coef0s:
        for C in Cs:
            acc = 0
            for dataset in train_and_valid:
                valid_acc = 0
                for train_data, train_labels, valid_data, valid_labels in dataset:
                    svm = SVC(kernel='sigmoid', gamma=gamma, coef0=coef0, C=C, max_iter=10_000)
                    svm.fit(train_data, train_labels)
                    valid_acc += svm.score(valid_data, valid_labels)
                acc += valid_acc / k
            global_accs.append(acc / k)
best_gamma = gammas[np.argmax(global_accs) // 100 ]
best_coef0 = coef0s[np.argmax(global_accs) % 100 // 10]
best_C = Cs[np.argmax(global_accs) % 10]
test_acc = 0
for data, labels, test_data, test_labels in rest_and_test:
    svm = SVC(kernel='sigmoid', gamma=best_gamma, coef0=best_coef0, C=C)
    svm.fit(data, labels)
    test_acc += svm.score(test_data, test_labels)
test_acc /= k
print(best_gamma)
print(best_coef0)
print(best_C)
print(test_acc)

1e-09
2
10000000
0.9279149200434714


In [ ]:
#Wygląda na to, że najlepiej dopasowują się kernele: wielomianowy oraz rbf.